## Identify the Apparels

In [ ]:
# Importing libraries
import os
import cv2
import math
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
# Importing model
from model import compute_cost, create_placeholders, forward_propagation, \
initializer_parameters, mini_batches_X, random_mini_batches

In [ ]:
# Loading .csv files
train = pd.read_csv('train/train.csv')
test = pd.read_csv('test/test.csv')

In [ ]:
X_train = train.iloc[:, 0].values
Y_train = train.iloc[:, 1].values
X_test = test.iloc[:, 0].values

del test, train

In [ ]:
# Converting output to OneHot Matrix
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
Y_train = onehot.fit_transform(Y_train[:, np.newaxis]).toarray()

In [ ]:
# Loading image files (28, 28, 4)
X_train_ = []
X_test_ = []

for i in tqdm(X_train):
    arr = cv2.imread(os.path.join('train', 'train', '%d.png' %(i)), -1)
    X_train_.append(arr)
X_train = np.asarray(X_train_, dtype = np.float32)

for i in tqdm(X_test):
    arr = cv2.imread(os.path.join('test', 'test', '%d.png' %(i)), -1)
    X_test_.append(arr)
X_test = np.asarray(X_test_, dtype = np.float32)

del i, X_test_, X_train_, arr

In [ ]:
# Saving numpy files
np.save(os.path.join('npy_files', "X_train.npy"), X_train)
np.save(os.path.join('npy_files', "Y_train.npy"), Y_train)
np.save(os.path.join('npy_files', "X_test.npy"), X_test)

In [ ]:
# Tensorflow model
m, n_H, n_W, n_C = X_train.shape
n_Y = Y_train.shape[1]
costs = []

In [ ]:
# #Resetting graph
tf.reset_default_graph()

In [ ]:
# Creating model
X, Y = create_placeholders(n_H, n_W, n_C, n_Y)
parameters = initializer_parameters()
Z = forward_propagation(X, parameters, reuse = None)
cost = compute_cost(Z, Y)
optimizer = tf.train.AdamOptimizer(learning_rate = 0.00025).minimize(cost)

In [ ]:
# Initializing session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [ ]:
# Training
for i in range(100):
    minibatch_cost = 0.
    num_batches = int(m / 2048)
    minibatches = random_mini_batches(X_train, Y_train, mini_batch_size= 2048)
    for minibatch in minibatches:
        (minibatch_X, minibatch_Y) = minibatch
        _, temp_cost = session.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
        minibatch_cost += temp_cost
    print("Cost after %d epochs = %f" %(i+1, minibatch_cost))
    if i%5 == 0:
        costs.append(minibatch_cost)

In [ ]:
# Getting parameters for the model
parameters = session.run(parameters)

In [ ]:
# Clearing memory
del X_test_, X_train_, arr, costs, count, i, m, minibatch, minibatch_X, \
minibatch_Y, minibatch_cost, minibatches, num_batches, temp_cost

In [ ]:
# Predictions on test set
Y_test_pred = session.run(forward_propagation(X_test, parameters, reuse = True))
Y_test_pred = session.run(tf.nn.softmax(Y_test_pred))
Y_test_pred = session.run(tf.contrib.seq2seq.hardmax(Y_test_pred))
Y_test_pred = np.array([np.where(r == 1)[0][0] for r in Y_test_pred])
Y_test_pred = Y_test_pred.reshape((Y_test_pred.shape[0], 1))

In [ ]:
output = {'id': np.arange(60001, 70001, 1),
          'label': Y_test_pred.reshape(Y_test_pred.shape[0])
          }

In [ ]:
# Saving outputs
df = pd.DataFrame(output, columns = ['id', 'label'])
df.to_csv(r'Output.csv', index = None, header = True)